In [48]:
!pip install sentence_transformers

Defaulting to user installation because normal site-packages is not writeable
     -------------------------------------- 340.6/340.6 kB 7.0 MB/s eta 0:00:00
     -------------------------------------- 204.1/204.1 MB 5.4 MB/s eta 0:00:00
     ------------------------------------- 481.2/481.2 kB 15.2 MB/s eta 0:00:00
     --------------------------------------- 10.2/10.2 MB 19.2 MB/s eta 0:00:00
     ---------------------------------------- 45.7/45.7 kB ? eta 0:00:00
     ------------------------------------- 194.4/194.4 kB 12.3 MB/s eta 0:00:00
     ---------------------------------------- 6.2/6.2 MB 18.8 MB/s eta 0:00:00
     ---------------------------------------- 2.4/2.4 MB 22.2 MB/s eta 0:00:00
     -------------------------------------- 308.9/308.9 kB 9.6 MB/s eta 0:00:00


  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


In [2]:
!pip install fuzzywuzzy

Defaulting to user installation because normal site-packages is not writeable


In [ ]:
import numpy as np
import pandas as pd

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

from sentence_transformers import SentenceTransformer
from fuzzywuzzy import fuzz


from transformers import pipeline
from sklearn.decomposition import LatentDirichletAllocation


In [4]:
%run "C:\Users\HALID\Documents\ITE Assignment\ITE-data-sciene-assignment\code\data_preprocessing.ipynb"

In [5]:
exhibitors=pd.read_csv("C:\\Users\\HALID\\Documents\\ITE Assignment\\ITE-data-sciene-assignment\\source\\exhibitors.csv")
exhibitor_categories=pd.read_csv("C:\\Users\\HALID\\Documents\\ITE Assignment\\ITE-data-sciene-assignment\\source\\exhibitor_categories.csv")

visitors=pd.read_csv("C:\\Users\\HALID\\Documents\\ITE Assignment\\ITE-data-sciene-assignment\\source\\visitors.csv")
visitors_questions=pd.read_csv("C:\\Users\\HALID\\Documents\\ITE Assignment\\ITE-data-sciene-assignment\\source\\visitors_questions.csv")
visitors_answers=pd.read_csv("C:\\Users\\HALID\\Documents\\ITE Assignment\\ITE-data-sciene-assignment\\source\\visitors_answers.csv")

In [6]:
exhibitor_agg,exhibitor_final=preprocess_exhibitor_categories(exhibitors,exhibitor_categories)

In [7]:
basedata,visitor_agg,visitor_raw=process_visitor_data(visitors,visitors_questions,visitors_answers)

In [8]:
basedata.rename(columns={"Please indicate your company's main area of business":"company_business"},inplace=True)

In [9]:
basedata["company_business"]=basedata["company_business"].fillna('')
exhibitor_agg['categoryName']=exhibitor_agg['categoryName'].fillna('')

In [10]:
all_text=pd.concat([basedata["company_business"],exhibitor_agg['categoryName']], axis=0)

In [11]:
vectorizer = TfidfVectorizer(stop_words='english')
tfidf_matrix = vectorizer.fit_transform(all_text)

In [12]:
visitor_embeddings = tfidf_matrix[:len(basedata)]
exhibitor_embeddings = tfidf_matrix[len(basedata):]

In [13]:
similarity_matrix = cosine_similarity(visitor_embeddings, exhibitor_embeddings)

In [14]:
recommendations = {}
for i, visitor_id in enumerate(basedata['id']):
    exhibitor_scores = list(enumerate(similarity_matrix[i]))
    exhibitor_scores = sorted(exhibitor_scores, key=lambda x: x[1], reverse=True)
    top_exhibitors = [(exhibitor_agg.iloc[idx]['exhibitorid'], score) for idx, score in exhibitor_scores]
    recommendations[visitor_id] = top_exhibitors

In [15]:
for visitor, matches in recommendations.items():
    print(f"Visitor {visitor} matches with Exhibitors:")
    for exhibitor, score in matches:
        print(f"  Exhibitor {exhibitor} with score {score:.2f}")
    print("\n")

Visitor 0wcaegyyobblhhvfzwyibn0a matches with Exhibitors:
  Exhibitor 90556 with score 0.63
  Exhibitor 30134 with score 0.36
  Exhibitor 92491 with score 0.32
  Exhibitor 92496 with score 0.30
  Exhibitor 92494 with score 0.29
  Exhibitor 17729 with score 0.26
  Exhibitor 31331 with score 0.17
  Exhibitor 21865 with score 0.15
  Exhibitor 92495 with score 0.12
  Exhibitor 54579 with score 0.11
  Exhibitor 92462 with score 0.10
  Exhibitor 98889 with score 0.08
  Exhibitor 10846 with score 0.00
  Exhibitor 13033 with score 0.00
  Exhibitor 15642 with score 0.00
  Exhibitor 15881 with score 0.00
  Exhibitor 18004 with score 0.00
  Exhibitor 18960 with score 0.00
  Exhibitor 27827 with score 0.00
  Exhibitor 32278 with score 0.00
  Exhibitor 44942 with score 0.00
  Exhibitor 46350 with score 0.00
  Exhibitor 48028 with score 0.00
  Exhibitor 51839 with score 0.00
  Exhibitor 55311 with score 0.00
  Exhibitor 56923 with score 0.00
  Exhibitor 68142 with score 0.00
  Exhibitor 72153 with s

### Hybrid approach with transformers + fuzzy logic + cosine similarity 

In [16]:
vectorizer = TfidfVectorizer(stop_words='english')
tfidf_matrix = vectorizer.fit_transform(pd.concat([basedata["company_business"], exhibitor_agg['categoryName']], axis=0))
visitor_embeddings_tfidf = tfidf_matrix[:len(basedata)]
exhibitor_embeddings_tfidf = tfidf_matrix[len(basedata):]

In [17]:
model = SentenceTransformer('all-MiniLM-L6-v2')
visitor_embeddings_bert = model.encode(basedata["company_business"], convert_to_tensor=True)
exhibitor_embeddings_bert = model.encode(exhibitor_agg['categoryName'], convert_to_tensor=True)

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

C:\Users\HALID\AppData\Roaming\Python\Python39\site-packages\huggingface_hub\file_download.py:144: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\HALID\.cache\huggingface\hub\models--sentence-transformers--all-MiniLM-L6-v2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [18]:
cosine_sim_tfidf = cosine_similarity(visitor_embeddings_tfidf, exhibitor_embeddings_tfidf)
cosine_sim_bert = cosine_similarity(visitor_embeddings_bert.cpu().numpy(), exhibitor_embeddings_bert.cpu().numpy())

In [19]:
def fuzzy_match(text1, text2):
    return fuzz.token_set_ratio(text1, text2) / 100.0

In [21]:
recommendations = {}
for i, visitor_id in enumerate(basedata['id']):
    exhibitor_scores = []
    for j, exhibitor_id in enumerate(exhibitor_agg['exhibitorid']):
        tfidf_score = cosine_sim_tfidf[i][j]
        bert_score = cosine_sim_bert[i][j]
        fuzzy_score = fuzzy_match(basedata.iloc[i]['company_business'], exhibitor_agg.iloc[j]['categoryName'])
        hybrid_score = (tfidf_score + bert_score + fuzzy_score) / 3  # Weighted Average
        exhibitor_scores.append((exhibitor_id, hybrid_score))
    exhibitor_scores = sorted(exhibitor_scores, key=lambda x: x[1], reverse=True)
    recommendations[visitor_id] = exhibitor_scores

In [22]:
for visitor, matches in recommendations.items():
    print(f"Visitor {visitor} matches with Exhibitors:")
    for exhibitor, score in matches:
        print(f"  Exhibitor {exhibitor} with score {score:.2f}")
    print("\n")

Visitor 0wcaegyyobblhhvfzwyibn0a matches with Exhibitors:
  Exhibitor 90556 with score 0.81
  Exhibitor 30134 with score 0.68
  Exhibitor 92491 with score 0.66
  Exhibitor 92496 with score 0.62
  Exhibitor 92494 with score 0.61
  Exhibitor 17729 with score 0.55
  Exhibitor 21865 with score 0.49
  Exhibitor 31331 with score 0.43
  Exhibitor 54579 with score 0.35
  Exhibitor 92495 with score 0.34
  Exhibitor 92462 with score 0.30
  Exhibitor 80700 with score 0.27
  Exhibitor 98889 with score 0.24
  Exhibitor 27827 with score 0.24
  Exhibitor 56923 with score 0.21
  Exhibitor 46350 with score 0.20
  Exhibitor 32278 with score 0.19
  Exhibitor 92492 with score 0.19
  Exhibitor 97818 with score 0.18
  Exhibitor 92493 with score 0.18
  Exhibitor 44942 with score 0.17
  Exhibitor 92497 with score 0.16
  Exhibitor 10846 with score 0.15
  Exhibitor 68142 with score 0.15
  Exhibitor 48028 with score 0.15
  Exhibitor 74870 with score 0.14
  Exhibitor 13033 with score 0.14
  Exhibitor 15881 with s

## Intent based recommendation

In [24]:
def extract_topics(corpus, num_topics=5, num_words=5):
    vectorizer = TfidfVectorizer(stop_words='english', max_features=1000)
    X = vectorizer.fit_transform(corpus)
    lda = LatentDirichletAllocation(n_components=num_topics, random_state=42)
    lda.fit(X)
    feature_names = vectorizer.get_feature_names_out()
    topics = []
    for topic_idx, topic in enumerate(lda.components_):
        topics.append(" ".join([feature_names[i] for i in topic.argsort()[:-num_words - 1:-1]]))
    return topics

In [25]:
intent_labels = extract_topics(basedata['company_business'])

# Zeroshot
classifier = pipeline("zero-shot-classification", model="facebook/bart-large-mnli")

config.json:   0%|          | 0.00/1.15k [00:00<?, ?B/s]

C:\Users\HALID\AppData\Roaming\Python\Python39\site-packages\huggingface_hub\file_download.py:144: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\HALID\.cache\huggingface\hub\models--facebook--bart-large-mnli. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regu

model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Device set to use cpu


In [27]:
def classify_intent(text):
    result = classifier(text, intent_labels)
    return result['labels'][0]

In [28]:
basedata['primary_intent'] = basedata[basedata['company_business']!=""]['company_business'].apply(classify_intent)

vectorizer = TfidfVectorizer(stop_words='english')
tfidf_matrix = vectorizer.fit_transform(pd.concat([basedata['company_business'], exhibitor_agg['categoryName']], axis=0))
visitor_embeddings_tfidf = tfidf_matrix[:len(basedata)]
exhibitor_embeddings_tfidf = tfidf_matrix[len(basedata):]

In [29]:
model = SentenceTransformer('all-MiniLM-L6-v2')
visitor_embeddings_bert = model.encode(basedata['company_business'], convert_to_tensor=True)
exhibitor_embeddings_bert = model.encode(exhibitor_agg['categoryName'], convert_to_tensor=True)

In [30]:
cosine_sim_tfidf = cosine_similarity(visitor_embeddings_tfidf, exhibitor_embeddings_tfidf)
cosine_sim_bert = cosine_similarity(visitor_embeddings_bert.cpu().numpy(), exhibitor_embeddings_bert.cpu().numpy())

In [31]:
def fuzzy_match(text1, text2):
    return fuzz.token_set_ratio(text1, text2) / 100.0

In [32]:
recommendations = {}
for i, visitor_id in enumerate(basedata['id']):
    exhibitor_scores = []
    for j, exhibitor_id in enumerate(exhibitor_agg['exhibitorid']):
        tfidf_score = cosine_sim_tfidf[i][j]
        bert_score = cosine_sim_bert[i][j]
        fuzzy_score = fuzzy_match(basedata.iloc[i]['company_business'], exhibitor_agg.iloc[j]['categoryName'])
        hybrid_score = (tfidf_score + bert_score + fuzzy_score) / 3  # Weighted Average
        exhibitor_scores.append((exhibitor_id, hybrid_score))
    exhibitor_scores = sorted(exhibitor_scores, key=lambda x: x[1], reverse=True)
    recommendations[visitor_id] = exhibitor_scores

In [33]:
for visitor, matches in recommendations.items():
    primary_intent = basedata.loc[basedata['id'] == visitor, 'primary_intent'].values[0]
    print(f"Visitor {visitor} ({primary_intent}) matches with Exhibitors:")
    for exhibitor, score in matches:
        print(f"  Exhibitor {exhibitor} with score {score:.2f}")
    print("\n")

Visitor 0wcaegyyobblhhvfzwyibn0a (agent travel tour operator management) matches with Exhibitors:
  Exhibitor 90556 with score 0.81
  Exhibitor 30134 with score 0.68
  Exhibitor 92491 with score 0.66
  Exhibitor 92496 with score 0.62
  Exhibitor 92494 with score 0.61
  Exhibitor 17729 with score 0.55
  Exhibitor 21865 with score 0.49
  Exhibitor 31331 with score 0.43
  Exhibitor 54579 with score 0.35
  Exhibitor 92495 with score 0.34
  Exhibitor 92462 with score 0.30
  Exhibitor 80700 with score 0.27
  Exhibitor 98889 with score 0.24
  Exhibitor 27827 with score 0.24
  Exhibitor 56923 with score 0.21
  Exhibitor 46350 with score 0.20
  Exhibitor 32278 with score 0.19
  Exhibitor 92492 with score 0.19
  Exhibitor 97818 with score 0.18
  Exhibitor 92493 with score 0.18
  Exhibitor 44942 with score 0.17
  Exhibitor 92497 with score 0.16
  Exhibitor 10846 with score 0.15
  Exhibitor 68142 with score 0.15
  Exhibitor 48028 with score 0.15
  Exhibitor 74870 with score 0.14
  Exhibitor 13033 

# Code deployment for recommendation

In [59]:


vectorizer = TfidfVectorizer(stop_words='english')
model = SentenceTransformer('all-MiniLM-L6-v2')
classifier = pipeline("zero-shot-classification", model="facebook/bart-large-mnli")

def extract_topics(corpus, num_topics=5, num_words=5):
    vectorizer = TfidfVectorizer(stop_words='english', max_features=1000)
    X = vectorizer.fit_transform(corpus)
    lda = LatentDirichletAllocation(n_components=num_topics, random_state=42)
    lda.fit(X)
    feature_names = vectorizer.get_feature_names_out()
    topics = []
    for topic_idx, topic in enumerate(lda.components_):
        topics.append(" ".join([feature_names[i] for i in topic.argsort()[:-num_words - 1:-1]]))
    return topics

def classify_intent(text, intent_labels):
    result = classifier(text, intent_labels)
    return result['labels'][0]

def fuzzy_match(text1, text2):
    return fuzz.token_set_ratio(text1, text2) / 100.0

def generate_recommendations(visitors, exhibitors):
    visitors['company_business'] = visitors['company_business'].fillna('')
    exhibitors['categoryName'] = exhibitors['categoryName'].fillna('')
    
    intent_labels = extract_topics(visitors['company_business'])
    visitors['primary_intent'] = basedata[basedata['company_business']!=""]['company_business'].apply(lambda x: classify_intent(x, intent_labels))
    
    tfidf_matrix = vectorizer.fit_transform(pd.concat([visitors['company_business'], exhibitors['categoryName']], axis=0))
    visitor_embeddings_tfidf = tfidf_matrix[:len(visitors)]
    exhibitor_embeddings_tfidf = tfidf_matrix[len(visitors):]
    
    visitor_embeddings_bert = model.encode(visitors['company_business'], convert_to_tensor=True)
    exhibitor_embeddings_bert = model.encode(exhibitors['categoryName'], convert_to_tensor=True)
    
    cosine_sim_tfidf = cosine_similarity(visitor_embeddings_tfidf, exhibitor_embeddings_tfidf)
    cosine_sim_bert = cosine_similarity(visitor_embeddings_bert.cpu().numpy(), exhibitor_embeddings_bert.cpu().numpy())
    
    recommendations = {}
    for i, visitor_id in enumerate(visitors['id']):
        exhibitor_scores = []
        for j, exhibitor_id in enumerate(exhibitors['exhibitorid']):
            tfidf_score = cosine_sim_tfidf[i][j]
            bert_score = cosine_sim_bert[i][j]
            fuzzy_score = fuzzy_match(visitors.iloc[i]['company_business'], exhibitors.iloc[j]['categoryName'])
            hybrid_score = (tfidf_score + bert_score + fuzzy_score) / 3  # Weighted Average
            exhibitor_scores.append((exhibitor_id, hybrid_score))
        exhibitor_scores = sorted(exhibitor_scores, key=lambda x: x[1], reverse=True)[:10]
        recommendations[visitor_id] = exhibitor_scores
    
    return recommendations



Device set to use cpu


In [64]:
exhibitor_agg.to_csv(r"C:\Users\HALID\Documents\ITE\exhibitor.csv")

In [ ]:
=generate_recommendations(basedata, exhibitor_agg)

{'0wcaegyyobblhhvfzwyibn0a': [(90556, 0.8086256773473433),
  (30134, 0.6789606696116239),
  (92491, 0.655101267636509),
  (92496, 0.6171833353710129),
  (92494, 0.6092216059457255),
  (17729, 0.5503840476067935),
  (21865, 0.4864978358351879),
  (31331, 0.4330067292896349),
  (54579, 0.34604414260920474),
  (92495, 0.33801340864914836)],
 '1o5g2tlsho4gxzk3usr06z1c': [(46350, 0.06754799683888753),
  (80700, 0.05808554093043009),
  (18004, 0.05561801294485728),
  (48028, 0.05309566855430603),
  (30134, 0.040788802007834114),
  (55311, 0.0395661915342013),
  (92496, 0.03493003795544306),
  (44942, 0.033586256206035614),
  (92497, 0.03148535887400309),
  (92495, 0.030912319819132488)],
 '35d97gdotgwa5lwpd0vs7k0h': [(97818, 0.4849871935477192),
  (31331, 0.4636953955345568),
  (32278, 0.4553455368963995),
  (92494, 0.4331315641618947),
  (72153, 0.41857165723567874),
  (92462, 0.41546229234442417),
  (10846, 0.39949205136704613),
  (27827, 0.3961244243844171),
  (44942, 0.3948551147777091),

the above code can even modify to get the recommendation for each visitor while registering also, instead of running as batch and the recommendation can be passed through API

### Unit test case

In [ ]:
class TestRecommendationSystem(unittest.TestCase):
    def setUp(self):
        self.visitors = pd.DataFrame({'id': [1], 'company_business': ["tourism, visa service"]})
        self.exhibitors = pd.DataFrame({'exhibitorid': [101, 102], 'categoryName': ["visa issuing authrity", "department of tourism"]})
    
    def test_extract_topics(self):
        topics = extract_topics(self.visitors['company_business'])
        self.assertTrue(len(topics) > 0)
    
    def test_classify_intent(self):
        intent_labels = ["tourism", "visa service"]
        intent = classify_intent("isa issuing authrity", intent_labels)
        self.assertIn(intent, intent_labels)
    
    def test_fuzzy_match(self):
        score = fuzzy_match("tourism", "department of tourism")
        self.assertTrue(0 <= score <= 1)
    
    def test_generate_recommendations(self):
        recommendations = generate_recommendations(self.visitors, self.exhibitors)
        self.assertTrue(1 in recommendations)
        self.assertTrue(len(recommendations[1]) > 0)